In [1]:
from behavior_and_time_alignment import beh_and_time_alignment
from session_preprocessing import ephys_opto_preprocessing, session_crosscorr, ephys_opto_crosscorr
from opto_tagging import opto_plotting_session, opto_plotting_unit
from opto_waveforms_preprocessing import opto_wf_preprocessing
from drift_analysis import plot_session_opto_drift
import matplotlib.pyplot as plt
from utils.beh_functions import session_dirs, get_unit_tbl
import numpy as np
import pandas as pd
import os
from utils.plot_utils import combine_pdf_big
import spikeinterface as si
from joblib import Parallel, delayed
import pickle

In [3]:
session_assets = pd.read_csv('/root/capsule/code/data_management/session_assets.csv')
session_list = session_assets['session_id']
session_list[-14:-3]

69    behavior_758018_2025-03-19_11-16-44
70    behavior_758018_2025-03-20_11-53-05
71    behavior_758018_2025-03-21_11-00-34
72    behavior_752014_2025-03-25_12-09-20
73    behavior_752014_2025-03-26_11-18-57
74    behavior_752014_2025-03-27_12-03-59
75    behavior_752014_2025-03-28_11-04-59
76    behavior_761038_2025-04-15_10-25-11
77    behavior_761038_2025-04-16_10-39-10
78    behavior_761038_2025-04-17_11-03-16
79    behavior_761038_2025-04-18_12-37-39
Name: session_id, dtype: object

In [4]:
# combine opto pdfs
data_type = 'curated'
def process(session, data_type):
    pdf_dir = session_dirs(session)[f'opto_dir_fig_{data_type}']
    output_pdf = os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_png.pdf')
    if os.path.exists(pdf_dir) and not os.path.exists(output_pdf):
        print(session)
        combine_pdf_big(pdf_dir, output_pdf)
Parallel(n_jobs=4)(delayed(process)(session, data_type) for session in session_list[-14:-3])
# for session in session_list[-14:-3]:
#     try:
#         process(session, data_type)
#         plt.close('all')
#     except:
#         print(f'Failed to process {session}')

behavior_758018_2025-03-19_11-16-44
behavior_752014_2025-03-25_12-09-20
behavior_758018_2025-03-20_11-53-05
behavior_758018_2025-03-21_11-00-34
Processing 202 files in /root/capsule/scratch/758018/behavior_758018_2025-03-19_11-16-44/ephys/opto/raw/figures
Processing 195 files in /root/capsule/scratch/752014/behavior_752014_2025-03-25_12-09-20/ephys/opto/raw/figures
Processing 157 files in /root/capsule/scratch/758018/behavior_758018_2025-03-20_11-53-05/ephys/opto/raw/figures
Processing 248 files in /root/capsule/scratch/758018/behavior_758018_2025-03-21_11-00-34/ephys/opto/raw/figures
Combined PDF saved as: /root/capsule/scratch/752014/behavior_752014_2025-03-25_12-09-20/ephys/opto/raw/behavior_752014_2025-03-25_12-09-20_opto_tagging_png.pdf
behavior_752014_2025-03-26_11-18-57
Processing 224 files in /root/capsule/scratch/752014/behavior_752014_2025-03-26_11-18-57/ephys/opto/raw/figures
Combined PDF saved as: /root/capsule/scratch/758018/behavior_758018_2025-03-21_11-00-34/ephys/opto/r

[None, None, None, None, None, None, None, None, None, None, None]

In [5]:
# from joblib import Parallel, delayed
# session_list = ['behavior_751181_2025-02-25_12-12-35',
#                 'behavior_751181_2025-02-26_11-51-19']
data_type = 'curated'
def process(session, data_type='raw'):
    plt.close('all')
    session_dir = session_dirs(session)
    if session_dir['curated_dir_curated'] is None:
        return None
    print(f'{session} start')
    beh_and_time_alignment(session) # plot behavior, check alignment between behavior, sound, and ephys
    plt.close('all')
    ephys_opto_preprocessing(session, data_type, 'soma') 
    # # redo spiketimes if not aligned, make opto_table (laser time in nidaq); 
    # # generate a pkl file with opto_resp/lat to each condition
    # plt.close('all')
    ephys_opto_crosscorr(session, data_type)
    # # separate session into opto/beh, cal cross-correlation
    # plt.close('all')
    opto_wf_preprocessing(session, data_type, 'soma', load_sorting_analyzer=True)
    # recal wf based on conditions. (pre/post, power), cal similarity, euclidean distance, correlation with spont
    # pay attention to difference early and late in the session
    
    opto_plotting_session(session, data_type, 'soma', plot=True, resp_thresh=0.3, lat_thresh=0.02, save=True)    
    # # save as pickle
    # with open(os.path.join(session_dirs(session)[f'opto_dir_{data_type}'], f'{session}_opto_tagging_metrics.pkl'), 'wb') as f:
    #     opto_tagging_df.to_pickle(f)
    
    # if '751004' in session:
    #     plot_session_opto_drift(session, data_type, update_csv=False)
    # else:     
    #     plot_session_opto_drift(session, data_type, update_csv=True)
    print(f'{session} stop')
        
 
Parallel(n_jobs=2)(delayed(process)(session, data_type = data_type) for session in session_list[-14:-3])

# for session in session_list[-14:-3]:
#     print(session)
#     session_dir = session_dirs(session)
#     # if session_dir['curated_dir_curated'] is not None:
#     process(session, data_type=data_type) 
#     # elif  session_dir['curated_dir_raw'] is not None:
#     #     print('Raw')
#     #     process(session, data_type='raw')
process('behavior_761038_2025-04-15_10-25-11')
        
 

behavior_758018_2025-03-20_11-53-05 start
behavior_758018_2025-03-19_11-16-44 start
behavior_758018_2025-03-19_11-16-44
behavior_758018_2025-03-20_11-53-05
behavior_758018_2025-03-20_11-53-05
behavior_758018_2025-03-19_11-16-44


compute_waveforms (workers: 5 processes):  91%|█████████ | 4444/4901 [21:22<04:01,  1.89it/s]/opt/conda/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/opt/conda/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
compute_waveforms (workers: 5 processes):  93%|█████████▎| 4552/4901 [22:35<04:09,  1.40it/s]/root/capsule/code/beh_ephys_analysis/opto_tagging.py:417: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptio

82 out of 154 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


/root/capsule/code/beh_ephys_analysis/opto_tagging.py:456: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  opto_wf = opto_wf.apply(pd.to_numeric, errors='ignore')
compute_waveforms (workers: 5 processes):  93%|█████████▎| 4567/4901 [22:44<03:18,  1.69it/s]/root/capsule/code/beh_ephys_analysis/opto_tagging.py:513: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  opto_tagging_df_sess_metrics = pd.concat([opto_tagging_df_sess_metrics, opto_tagging_df_curr], ignore_index=True) # tidy
compute_waveforms (workers: 5 processes):  93%|█████████▎| 4573/4901 [22:48<03:41,  1.48it/s]/root/capsule/code/beh_ephys_analysis/opto_tagging.py:513: F

UnboundLocalError: local variable 'peak_waveform_spont' referenced before assignment

In [4]:
process('behavior_758018_2025-03-19_11-16-44', data_type='raw')

behavior_758018_2025-03-19_11-16-44 start
107 out of 201 units pass quality control
{'samples_to_keep': [-30, 60], 'y_neighbors_to_keep': 3, 'orginal_loc': False}


KeyboardInterrupt: 

In [ ]:
# make a df from all_Tm, columns correspond to focus regressors
 